In [ ]:
import sys
sys.path.insert(0, '/work/greenhouse-simulator-2/')

from datetime import datetime, timezone
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import simps
from numpy import trapz

from helpers.types import *
from helpers.data_prep import *
from helpers.solar_conversions import *
from greenhouse.structure.structure import *

In [ ]:
def get_azimuth_angles_of_faces(faces_on_full_circle=12):
    assert faces_on_full_circle % 2 == 0, "Number of faces needs to be even"

    angle_of_face: deg = 360 / faces_on_full_circle

    azimuth_angles_of_faces = []
    for i in range(int(faces_on_full_circle / 2)):
        azimuth_of_face = angle_of_face / 2 + i * angle_of_face
        azimuth_angles_of_faces.append(int(azimuth_of_face))

    return azimuth_angles_of_faces

azimuths_of_faces = get_azimuth_angles_of_faces(120)

In [ ]:
sunlight: umol_per_m2_s = 1800
led_light: umol_per_m2_s = 250

rotation_period_time: s = 300

latitude = 38.7436883
longitude = -9.1952227

timestamp = datetime(2020, 6, 22, 13, 40, 0, tzinfo=timezone.utc)

In [ ]:
coordinates = { "latitude": latitude, "longitude": longitude }

In [ ]:
data = []

for azimuth in azimuths_of_faces:
    int_coeff = get_intensity_coeff(coordinates, timestamp, 90, 270 - azimuth)
    data.append({
        "azimuth": azimuth,
        "intensity": int_coeff
    })

df = pd.DataFrame(data)

df.plot.line("azimuth", "intensity")

## Pre-generate curve

In [ ]:
def create_sine(xmax, ymax, x_offset=0, resolution=1000):
    """
    Creates a 'peak' of a sine function (part above the midline, pi period).

    Parameters
    ----------
    xmax : float
        Length of the period (relative to x_offset).
    ymax : float
        Y value of peak.
    x_offset : float
        Offset value along X axis.
    resolution : int
        Number of points within one peak.
    """

    x = np.arange(0, xmax, 1/resolution)
    y = np.sin(math.pi * x / xmax) * ymax

    x += x_offset

    return x, y

def pad_sine(y_curve, x_length, x_start, resolution=1000):
    """
    Stretches X axis (filled with zeros) to `x_length`, then adds the passed curve between `x_start`
    and `x_start + x_length`.

    Parameters
    ----------
    y_curve : np.array
        Curve to be padded.
    x_length : float
        Total length of the X axis.
    x_start : float
        Value on X axis where `y_curve` should start.
    resolution : int
        Number of points within one peak.
    """

    # Handle case if x_start negative
    y_curve_first_half = []
    if x_start < 0:
        y_curve_first_half = y_curve[0:int(abs(x_start))+1]
        y_curve = y_curve[int(abs(x_start)):]
        x_start = 0

    # Create a horizontal line at y=0 from x=0 to x=x_length
    x = np.arange(0, x_length, 1 / resolution)
    y = np.zeros(x_length * resolution)

    # Replace part of the line between the specified coordinates
    x_start_idx = round(resolution * x_start)
    x_end_idx = x_start_idx + len(y_curve)
    y[x_start_idx:x_end_idx] = y_curve

    if len(y_curve_first_half):
        y[-len(y_curve_first_half):] = y_curve_first_half

    return x, y

In [ ]:
def create_exposure_curves(curves, rotation_period, resolution=1000):
    """
    Creates curves that represent light exposure as function of rotation.

    Parameters
    ----------
    curves : dict
        Dict keys are curve names, values are a tuple containing (xmin, xlen, ymax,)
    rotation_period : s
        Length of one rotation.
    resolution : int
        Number of points within one peak.
    """

    def angle_to_sec(angle: deg) -> s:
        return angle / 360 * rotation_period

    results = { 
        "total": { 
            "x": np.arange(0, rotation_period, 1 / resolution), 
            "y": np.zeros(rotation_period * resolution)
        }
    }

    # Create individual exposure curves
    for curve_name in curves:
        xmin, xlen, ymax = curves[curve_name]

        # Convert from angles to seconds
        xmin = angle_to_sec(xmin)
        xlen = angle_to_sec(xlen)
        # Create base sine curve
        x, y = create_sine(xlen, ymax, resolution=resolution)

        # Extend X axis to full period length and slice in the sine curve
        x, y = pad_sine(y, x_length=rotation_period, x_start=xmin, resolution=resolution)

        results[curve_name] = {"x": x, "y": y}

    # Sum all curves to create `total`
    for result_name in results:
        result = results[result_name]
        results["total"]["y"] += result["y"]

    return results

In [ ]:
def get_curve_configs(
    sun_ppfd: umol_per_m2=None,
    led_ppfd: umol_per_m2=None,
    rotation_period: s=600,
    angle_exposed_to_sun: deg=180,
    angle_exposed_to_single_led: deg=90,
    inter_led_angle: deg=45,
    resolution=1000,
):
    """
    Creates the superposition of exposure curves of each individual light source to get
    the total irradiance of a plant.

    Parameters
    ----------
    sun_ppfd : umol_per_m2 | umol_per_m2[]
        Irradiance of sunlight (on a vertical surface perpendicular to the suns rays).
        If a list, length should equal to rotation_period * resolution.
    led_ppfd : umol_per_m2 | umol_per_m2[]
        Irradiance from a single light fixture. 
        If a list, length should equal to rotation_period * resolution.
    rotation_period : s
        Time it takes for the barrel to do a full rotation.
    angle_exposed_to_sun : deg
        Central angle between 2 radii that points to the start and end points of the barrel surface that
        is exposed to the sun.
    angle_exposed_to_single_led : deg
        Central angle between 2 radii that points to the start and end points of the barrel surface that
        is exposed to light from a single fixture.
    inter_led_angle : deg
        Angle between 2 radii that points to the center points of 2 adjacent light fixtures.
    resolution : int
        Number of points within one peak.
    """

    sun_azimuth: deg = 0

    sun_start: deg = sun_azimuth - angle_exposed_to_sun / 2
    led_1_start: deg = 180 - inter_led_angle - angle_exposed_to_single_led / 2
    led_2_start: deg = led_1_start + inter_led_angle
    led_3_start: deg = led_2_start + inter_led_angle

    curves = {
        "sun": [sun_start, angle_exposed_to_sun],
        "led_1": [led_1_start, angle_exposed_to_single_led],
        "led_2": [led_2_start, angle_exposed_to_single_led],
        "led_3": [led_3_start, angle_exposed_to_single_led]
    }

    # If PPFD values are defined, add them as a 3rd element
    if sun_ppfd and led_ppfd:
        for curve_name in curves:
            ppfd = led_ppfd if "led" in curve_name else sun_ppfd
            curves[curve_name] = [*curves[curve_name], ppfd]

    return curves

In [ ]:
def plot_curves(results):
    for result_name in results:
        if result_name != "total":
            result = results[result_name]
            plt.plot(result["x"], result["y"], label=result_name)

    plt.plot(results["total"]["x"], results["total"]["y"], label="total", linewidth=3)

    plt.legend()
    plt.show()

In [ ]:
rotation_period: s = 600
resolution = 1
curves = get_curve_configs(
    sun_ppfd=1000,
    led_ppfd=300,
    rotation_period=rotation_period, 
    resolution=1
)
results = create_exposure_curves(curves, rotation_period, resolution=resolution)


plot_curves(results)

In [ ]:
val1 = []
val2 = []
total = []
values = results["total"]["y"]
quart_len = int(len(values) / 4)
for i in range(2*quart_len):
    idx_1 = 3 * quart_len + i
    if idx_1 >= len(values):
        idx_1 = idx_1 - len(values)
    idx_2 = quart_len + i
    pt_1 = values[idx_1]
    pt_2 = values[idx_2]
    val1.append(pt_1)
    val2.append(pt_2)
    total.append(pt_1 + pt_2)

plt.plot(val1)
plt.plot(val2)
plt.plot(total)

### Set up daily light plan

In [ ]:
def calc_lighting_plan(photoperiod_h=14, ramp_length_h=1, period_length: s=600):
    photoperiod: s = photoperiod_h * 60 * 60
    ramp_length: s = ramp_length_h * 60 * 60

    period_count = int(photoperiod / period_length)
    ramp_period_count = int(ramp_length / period_length)
    const_period_count = period_count - 2 * ramp_period_count

    x_on = np.arange(0, ramp_period_count, 1 / ramp_period_count)
    y_on = np.sin(math.pi * x_on / ramp_period_count / 2)

    x_const = np.arange(ramp_period_count, ramp_period_count + const_period_count, 1 / const_period_count)
    y_const = np.ones(len(x_const))

    x_off = np.arange(ramp_period_count + const_period_count, 2 * ramp_period_count + const_period_count, 1 / ramp_period_count)
    y_off = np.sin(math.pi * x_off / ramp_period_count / 2)

    return x_on, y_on, x_const, y_const, x_off, y_off

In [ ]:
period_length: s = 600

x_on, y_on, x_const, y_const, x_off, y_off = calc_lighting_plan(period_length=period_length)

plt.figure(figsize=(16,9))
plt.plot(x_on, y_on)
plt.plot(x_const, y_const)
plt.plot(x_off, y_off)


In [ ]:
(sum(y_on * period_length) + sum(y_const * period_length) + sum(y_off * period_length)) / 1e6

In [ ]:
y_on

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>